In [ ]:
import os

import numpy as np
from PIL import Image


def load_image_from_folders(folder_path, label):
    images = []
    labels = []
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = Image.open(img_path).convert('RGB')
        # 缩放
        img_resized = img.resize((64, 64), Image.Resampling.LANCZOS)
        # 转换为numpy数组并归一化
        img_nd_array = np.array(img_resized) / 255.0
        images.append(img_nd_array.flatten())
        labels.append(label)
    images_nd_array = np.array(images)
    labels_nd_array = np.array(labels)
    return images_nd_array.T, labels_nd_array.T


# 基本sigmoid函数
def basic_sigmoid(x):
    return 1 / (1 + np.exp(-x))


# 对sigmoid函数求导
def sigmoid_derivative(x):
    res = basic_sigmoid(x)
    return res * (1 - res)


# 初始化网络参数
def initialize_with_zeros(shape):
    w = np.zeros((shape, 1))
    b = 0
    return w, b


# 前向和反向传播
def propagate(w, b, X, Y):
    # 样本数量
    m = X.shape[1]
    # 前向传播
    # w(n,1) X(n,m)
    A = basic_sigmoid(np.dot(w.T, X) + b)
    # 计算损失
    cost = -1 / m * np.sum(Y * np.log(A) + (1 - Y) * np.log(1 - A))
    # 反向传播
    dz = A - Y
    dw = 1 / m * np.dot(X, dz.T)
    db = 1 / m * np.sum(dz)
    cost = np.squeeze(cost)
    grads = {"dw": dw, "db": db}
    return grads, cost


def optimize(w, b, X, Y, num_iterations, learning_rate):
    costs = []
    for i in range(num_iterations):
        grads, cost = propagate(w, b, X, Y)
        dw = grads["dw"]
        db = grads["db"]
        w = w - learning_rate * dw
        b = b - learning_rate * db
        if i % 100 == 0:
            costs.append(cost)
            print("损失结果 %i: %f" % (i, cost))
    params = {"w": w, "b": b}
    grads = {"dw": dw, "db": db}
    return params, grads, costs


def predict(w, b, X):
    m = X.shape[1]
    Y_predict = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    A = basic_sigmoid(np.dot(w.T, X) + b)
    for i in range(A.shape[1]):
        if A[0, i] <= 0.5:
            Y_predict[0, i] = 0
        else:
            Y_predict[0, i] = 1
    return Y_predict


if __name__ == '__main__':
    # 加载训练集数据
    cat_train_X, cat_train_Y = load_image_from_folders(
        './data/cat/train', 1)
    dog_train_X, dog_train_Y = load_image_from_folders(
        './data/dog/train', 0)
    train_X = np.concatenate([cat_train_X, dog_train_X], axis=1)
    train_Y = np.concatenate([cat_train_Y, dog_train_Y])
    # 初始化参数
    w, b = initialize_with_zeros(train_X.shape[0])
    # 剃度下降
    # params:更新后的网络参数
    # grads:最后一次梯度
    # costs:每次更新损失列表
    params, grads, costs = optimize(w, b, train_X, train_Y, 3000, 0.001)
    # 获取训练的参数
    w = params["w"]
    b = params["b"]
    # 获取训练集的预测结果
    y_prediction_train = predict(w, b, train_X)
    # 打印准确率
    print('训练集准确率: {}'.format(100 - np.mean(np.abs(y_prediction_train - train_Y)) * 100))
    # 加载测试集数据
    test_cat_X, test_cat_Y = load_image_from_folders(
        './data/cat/test', 1)
    test_dog_X, test_dog_Y = load_image_from_folders(
        './data/dog/test', 0)
    test_X = np.concatenate([test_cat_X, test_dog_X], axis=1)
    test_Y = np.concatenate([test_cat_Y, test_dog_Y])
    y_prediction_test = predict(w, b, test_X)
    print('测试集准确率: {}'.format(100 - np.mean(np.abs(y_prediction_test - test_Y)) * 100))
